In [1]:
import pandas as pd
from lxml import html
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)


C:\Users\Masha\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Users\Masha\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [54]:
corpus_xml = html.fromstring(open('paraphraser/paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data_2 = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [55]:
data_2['text_1_norm'] = data_2['text_1'].apply(normalize)
data_2['text_2_norm'] = data_2['text_2'].apply(normalize)

In [6]:
data.head()

,text_1,text_2,label,text_1_norm,text_2_norm
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия


In [56]:
corpus_xml = html.fromstring(open('paraphraser/corpus.xml', 'rb').read())
texts = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//sentence'):
    texts.append(p.xpath('./value[@name="text"]/text()')[0])

    
data_train_2 = pd.DataFrame({'text':texts})

In [58]:
data_train_2['text_norm'] = data_train_2['text'].apply(normalize)

In [9]:
data_train.head()

,text,text_norm
0,"Избежать ""фискального обрыва"": Сенат США подде...",избежать фискальный обрыв сенат сша поддержать...
1,"""Фискальный обрыв"" в США временно предотвращен.",фискальный обрыв сша временно предотвратить
2,Чечня попросила националистов составить кодекс...,чечня попросить националист составить кодекс п...
3,Северокорейский лидер впервые за 19 лет поздра...,северокорейский лидер впервые 19 год поздравит...
4,В Кот-Д`Ивуаре десятки человек погибли в давке...,кот-д`ивуаре десятка человек погибнуть давка н...


## Матричные разложения

In [59]:
cv = CountVectorizer(min_df=3, max_df=0.4, max_features=2000) #для первой модели max_features=1000
tv = TfidfVectorizer(min_df=3, max_df=0.4, max_features=2000)

text_cv = cv.fit_transform(data_train['text_norm'])
text_tv = tv.fit_transform(data_train['text_norm'])

In [60]:
text_tv.shape

(12062, 2000)

In [61]:
id2word_cv = {i:w for i,w in enumerate(cv.get_feature_names())}
word2id_cv = {w:i for i,w in id2word_cv.items()}

id2word_tv = {i:w for i,w in enumerate(tv.get_feature_names())}
word2id_tv = {w:i for i,w in id2word_tv.items()}

## SVD и NMF

In [62]:
svd_tv = TruncatedSVD(100) #для первой модели n_components=50
svd_cv = TruncatedSVD(100)

nmf_tv = NMF(100) #для первой модели n_components=50
nmf_cv = NMF(100)

In [63]:
svd_tv.fit(text_tv)
svd_cv.fit(text_cv)

nmf_tv.fit(text_tv)
nmf_cv.fit(text_cv)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=100, random_state=None, shuffle=False, solver='cd',
  tol=0.0001, verbose=0)

In [64]:
id2vec_nmf_cv = nmf_cv.components_.T
id2vec_nmf_tv = nmf_cv.components_.T

In [65]:
id2vec_svd_cv = svd_cv.components_.T
id2vec_svd_tv = svd_tv.components_.T

In [66]:
data_2['text_1_svd_cv'] = list(map(np.array, svd_cv.transform(cv.transform(data_2['text_1_norm']))))
data_2['text_2_svd_cv'] = list(map(np.array, svd_cv.transform(cv.transform(data_2['text_2_norm']))))
data_2['text_1_svd_tv'] = list(map(np.array, svd_tv.transform(tv.transform(data_2['text_1_norm']))))
data_2['text_2_svd_tv'] = list(map(np.array, svd_tv.transform(tv.transform(data_2['text_2_norm']))))

data_2['text_1_nmf_cv'] = list(map(np.array, nmf_cv.transform(cv.transform(data_2['text_1_norm']))))
data_2['text_2_nmf_cv'] = list(map(np.array, nmf_cv.transform(cv.transform(data_2['text_2_norm']))))
data_2['text_1_nmf_tv'] = list(map(np.array, nmf_tv.transform(tv.transform(data_2['text_1_norm']))))
data_2['text_2_nmf_tv'] = list(map(np.array, nmf_tv.transform(tv.transform(data_2['text_2_norm']))))

## Word2Vec и Fastext

In [67]:
fast_text = gensim.models.FastText([text.split() for text in data_train['text']], size=100, min_n=4, max_n=8) #для первой модели size=50
fast_text_norm = gensim.models.FastText([text.split() for text in data_train['text_norm']], size=100, min_n=4, max_n=8)

In [68]:
w2v = gensim.models.Word2Vec([text.split() for text in data_train['text_norm']], size=100, sg=1) #для первой модели size=50

In [69]:
def get_embedding(text, model, dim, idf=False):
    text = text.split()
    
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i, word in enumerate(words):
        try:
            if idf:
                _idf = tv.idf_[word2id_tv[word]]
                v = model[word] / _idf
                vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
            else:
                v = model[word]
                vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector


### Word2Vec

In [72]:
dim = 100 #для первой модели dim=50
text_1_w2v = np.zeros((len(data_2['text_1_norm']), dim))
text_2_w2v = np.zeros((len(data_2['text_2_norm']), dim))

for i, text in enumerate(data_2['text_1_norm'].values):
    text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(data_2['text_2_norm'].values):
    text_2_w2v[i] = get_embedding(text, w2v, dim)

C:\Users\Masha\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [73]:
data_2['text_1_w2v'] = list(map(np.array, text_1_w2v))
data_2['text_2_w2v'] = list(map(np.array, text_2_w2v))

In [75]:
dim = 100 #для первой модели dim=50
text_1_w2v_idf = np.zeros((len(data_2['text_1_norm']), dim))
text_2_w2v_idf = np.zeros((len(data_2['text_2_norm']), dim))

for i, text in enumerate(data_2['text_1_norm'].values):
    text_1_w2v_idf[i] = get_embedding(text, w2v, dim, idf=True)
    
for i, text in enumerate(data_2['text_2_norm'].values):
    text_2_w2v_idf[i] = get_embedding(text, w2v, dim, idf=True)

C:\Users\Masha\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [76]:
data_2['text_1_w2v_idf'] = list(map(np.array, text_1_w2v_idf))
data_2['text_2_w2v_idf'] = list(map(np.array, text_2_w2v_idf))

### Fast Text

##### -norm -idf

In [79]:
dim = 100 #для первой модели dim=50
data_2['text_1_notnorm'] = data_2['text_1'].apply(tokenize)
data_2['text_2_notnorm'] = data_2['text_2'].apply(tokenize)

text_1_ft = np.zeros((len(data_2['text_1_notnorm']), dim))
text_2_ft = np.zeros((len(data_2['text_2_notnorm']), dim))

for i, text in enumerate(data_2['text_1_notnorm'].values):
    text_1_ft[i] = get_embedding(text, fast_text, dim)
    
for i, text in enumerate(data_2['text_2_notnorm'].values):
    text_2_ft[i] = get_embedding(text, fast_text, dim)

C:\Users\Masha\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [80]:
data_2['text_1_ft'] = list(map(np.array, text_1_ft))
data_2['text_2_ft'] = list(map(np.array, text_2_ft))

##### -norm +idf

In [81]:
dim = 100 #для первой модели dim=50
data_2['text_1_notnorm'] = data_2['text_1'].apply(tokenize)
data_2['text_2_notnorm'] = data_2['text_2'].apply(tokenize)

text_1_ft_idf = np.zeros((len(data_2['text_1_notnorm']), dim))
text_2_ft_idf = np.zeros((len(data_2['text_2_notnorm']), dim))

for i, text in enumerate(data_2['text_1_notnorm'].values):
    text_1_ft_idf[i] = get_embedding(text, fast_text, dim, idf=True)
    
for i, text in enumerate(data_2['text_2_notnorm'].values):
    text_2_ft_idf[i] = get_embedding(text, fast_text, dim, idf=True)

C:\Users\Masha\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [82]:
data_2['text_1_ft_idf'] = list(map(np.array, text_1_ft_idf))
data_2['text_2_ft_idf'] = list(map(np.array, text_2_ft_idf))

##### +norm -idf

In [83]:
dim = 100 #для первой модели dim=50
text_1_ft_norm = np.zeros((len(data_2['text_1_norm']), dim))
text_2_ft_norm = np.zeros((len(data_2['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    text_1_ft_norm[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    text_2_ft_norm[i] = get_embedding(text, w2v, dim)

C:\Users\Masha\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [84]:
data_2['text_1_ft_norm'] = list(map(np.array, text_1_ft_norm))
data_2['text_2_ft_norm'] = list(map(np.array, text_2_ft_norm))

##### +norm +idf

In [85]:
dim = 100 #для первой модели dim=50
text_1_ft_norm_idf = np.zeros((len(data_2['text_1_norm']), dim))
text_2_ft_norm_idf = np.zeros((len(data_2['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    text_1_ft_norm_idf[i] = get_embedding(text, w2v, dim, idf=True)
    
for i, text in enumerate(data['text_2_norm'].values):
    text_2_ft_norm_idf[i] = get_embedding(text, w2v, dim, idf=True)

C:\Users\Masha\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [86]:
data_2['text_1_ft_norm_idf'] = list(map(np.array, text_1_ft_norm_idf))
data_2['text_2_ft_norm_idf'] = list(map(np.array, text_2_ft_norm_idf))

In [33]:
data.head()

,text_1,text_2,label,text_1_norm,text_2_norm,text_1_svd_cv,text_2_svd_cv,text_1_svd_tv,text_2_svd_tv,text_1_nmf_cv,...,text_1_notnorm,text_2_notnorm,text_1_ft,text_2_ft,text_1_ft_idf,text_2_ft_idf,text_1_ft_norm,text_2_ft_norm,text_1_ft_norm_idf,text_2_ft_norm_idf
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...,"[0.04166197922250118, 0.02496725508033569, 0.0...","[0.0477085401442907, 0.03237978532086951, 0.04...","[0.03664657817408758, -0.024093505938884756, 0...","[0.04366220679045339, -0.03114908901681982, 0....","[0.001043527211797704, 0.0, 0.0022512963311124...",...,полицейским разрешат стрелять на поражение по ...,полиции могут разрешить стрелять по хулиганам ...,"[-0.03865686948928568, 0.1279540140595701, -0....","[-0.03509345301426947, 0.117421620991081, -0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.000292273674858734, 0.0009786967420950532,...","[0.007344291002179186, 0.03571530928214391, 0....","[0.006956571945920587, 0.034042839581767716, 0...","[0.0006283966358751059, 0.0038536646558592715,...","[0.000672562513500452, 0.0038367658077428737, ..."
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...,"[0.043754299198012506, 0.0312339495419738, 0.0...","[0.01625040088227963, 0.02468327885353348, 0.0...","[0.0307582724730046, -0.02495475935768012, 0.0...","[0.027649100521362394, -0.04453181611638255, 0...","[0.001491027045828315, 0.000706692623219274, 0...",...,право полицейских на проникновение в жилище ре...,правила внесудебного проникновения полицейских...,"[-0.047261725761927664, 0.15534284454770386, -...","[-0.04726228330816541, 0.15695655718445778, -0...","[-0.0004064256063429639, 0.0013025016523897648...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.002890786233668526, 0.031098345294594765, 0...","[0.00022549934995671114, 0.013413367172082266,...","[0.00039510004959690076, 0.00495685962960124, ...","[-0.00023174937814474106, 0.001414964285989602..."
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...,"[0.06410892156233608, 0.02219843259261879, 0.0...","[0.06702673813015111, 0.02458233689675373, 0.0...","[0.03902877441443022, -0.014949408659881102, 0...","[0.040496311864370145, -0.0165516359646364, 0....","[0.0, 0.0, 0.0, 0.0, 0.004782156304455325, 0.0...",...,президент египта ввел чрезвычайное положение в...,власти египта угрожают ввести в стране чрезвыч...,"[-0.03200452553573996, 0.10683710395824164, -0...","[-0.03140027498011477, 0.10463804204482585, -0...","[-0.0011024044652003795, 0.003675282117910683,...","[-0.0006003696471452713, 0.0020107616437599063...","[0.008035568121288503, 0.03112484168793474, 0....","[0.009737854690424033, 0.03710759963308062, 0....","[0.0011519782710820436, 0.004236071969249419, ...","[0.0012638938746282033, 0.004599628371319601, ..."
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия,"[0.05673181777383862, 0.012120000256676866, 0....","[0.11516093568207753, 0.05086460339415042, 0.0...","[0.040465792844098236, -0.009114661035387222, ...","[0.07761632100149807, -0.03681816326720013, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,вернувшихся из сирии россиян волнует вопрос тр...,самолеты мчс вывезут россиян из разрушенной сирии,"[-0.0295524710478882, 0.0976116903540161, -0.0...","[-0.025648014087762152, 0.08489384076425008, -...","[-0.0014830571340603961, 0.004967074065158765,...","[-0.002079600202185767, 0.006974594401461738, ...","[0.011316

In [87]:
data_2.head()

,text_1,text_2,label,text_1_norm,text_2_norm,text_1_svd_cv,text_2_svd_cv,text_1_svd_tv,text_2_svd_tv,text_1_nmf_cv,...,text_1_notnorm,text_2_notnorm,text_1_ft,text_2_ft,text_1_ft_idf,text_2_ft_idf,text_1_ft_norm,text_2_ft_norm,text_1_ft_norm_idf,text_2_ft_norm_idf
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...,"[0.04474610390283675, 0.026270221133088897, 0....","[0.0491953873797517, 0.03389992984896997, 0.04...","[0.03345577633683747, 0.009252940335128692, 0....","[0.041353928640877, 0.010296951688637751, 0.03...","[0.000454234770874703, 0.0, 0.0002991493535475...",...,полицейским разрешат стрелять на поражение по ...,полиции могут разрешить стрелять по хулиганам ...,"[0.006401477239301635, -0.017320871560109988, ...","[0.005927926293225028, -0.015788819699082524, ...","[0.00010683848328577976, -0.000306837157242827...","[0.00010803409895743243, -0.000297521095490083...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0005931832614199569, 0.0037425896540905037,...","[0.0005267075806235274, 0.0032485320698469877,..."
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...,"[0.04409420073349188, 0.03107435919735345, 0.0...","[0.016504129123378956, 0.02466260844351228, 0....","[0.04002491598342103, 0.010771804907766656, 0....","[0.04483163956889629, -0.004757068295173955, 0...","[0.0008669378864179647, 0.0002962282404319136,...",...,право полицейских на проникновение в жилище ре...,правила внесудебного проникновения полицейских...,"[0.007707247306825593, -0.021479111863300204, ...","[0.007887978846805968, -0.022089849093130658, ...","[7.065643512760289e-05, -0.0001905579774756916...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0002530181761054943, 0.0034056000877171755,...","[-0.00011798625928349793, 0.000946814659982919..."
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...,"[0.06557762543427555, 0.022931376437178873, 0....","[0.06847954686419644, 0.025211193690999754, 0....","[0.037429609322977384, 0.022308610210040943, 0...","[0.0390962563770806, 0.018353606223361137, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,президент египта ввел чрезвычайное положение в...,власти египта угрожают ввести в стране чрезвыч...,"[0.0054879100862308405, -0.014971666983910836,...","[0.005396100110374391, -0.014662657427834347, ...","[0.00019400263772695325, -0.000521865222253836...","[9.692605817690492e-05, -0.0002696036681300029...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0007023831463551947, 0.0034712449580963168,...","[0.0007789237591038857, 0.003661062867779817, ..."
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия,"[0.05975613884836182, 0.011269556404577462, 0....","[0.12205510338188756, 0.057299399505724304, 0....","[0.03911189630233726, 0.018031547110906345, -0...","[0.07327220895739607, 0.00046988122394741694, ...","[7.442845657754801e-05, 0.0, 0.0, 0.0008421906...",...,вернувшихся из сирии россиян волнует вопрос тр...,самолеты мчс вывезут россиян из разрушенной сирии,"[0.0047779868731999565, -0.013337420299649239,...","[0.004315683551664863, -0.011962969920464925, ...","[0.00024221827495946654, -0.000666306024262060...","[0.0003470510710030794, -0.0009341504690902573...","[0.0, 0.0

## Косинусная близость

In [35]:
def get_dicstance(vec1, vec2):
    return cosine_distances(vec1.reshape(1, -1), vec2.reshape(1, -1))[0][0]

In [36]:
X = pd.DataFrame()

In [89]:
X_2 = pd.DataFrame()

In [90]:
X_2['text_svd_cv'] = [get_dicstance(data_2['text_2_svd_cv'][i], v) for i, v in enumerate(data_2['text_1_svd_cv'])]
X_2['text_svd_tv'] = [get_dicstance(data_2['text_2_svd_tv'][i], v) for i, v in enumerate(data_2['text_1_svd_tv'])]
X_2['text_nmf_cv'] = [get_dicstance(data_2['text_2_nmf_cv'][i], v) for i, v in enumerate(data_2['text_1_nmf_cv'])]
X_2['text_nmf_tv'] = [get_dicstance(data_2['text_2_nmf_tv'][i], v) for i, v in enumerate(data_2['text_1_nmf_tv'])]
X_2['text_w2v'] = [get_dicstance(data_2['text_2_w2v'][i], v) for i, v in enumerate(data_2['text_1_w2v'])]
X_2['text_w2v_idf'] = [get_dicstance(data_2['text_2_w2v_idf'][i], v) for i, v in enumerate(data_2['text_1_w2v_idf'])]
X_2['text_ft'] = [get_dicstance(data_2['text_2_ft'][i], v) for i, v in enumerate(data_2['text_1_ft'])]
X_2['text_ft_idf'] = [get_dicstance(data_2['text_2_ft_idf'][i], v) for i, v in enumerate(data_2['text_1_ft_idf'])]
X_2['text_ft_norm'] = [get_dicstance(data_2['text_2_ft_norm'][i], v) for i, v in enumerate(data_2['text_1_ft_norm'])]
X_2['text_ft_norm_idf'] = [get_dicstance(data_2['text_2_ft_norm_idf'][i], v) for i, v in enumerate(data_2['text_1_ft_norm_idf'])]

In [38]:
y = data.label

In [39]:
X

,text_svd_cv,text_svd_tv,text_nmf_cv,text_nmf_tv,text_w2v,text_w2v_idf,text_ft,text_ft_idf,text_ft_norm,text_ft_norm_idf
0,1.142438,0.983126,0.983720,0.986670,0.000640,0.001735,3.904016e-06,1.000000e+00,0.000640,0.001735
1,0.009787,0.018832,0.031264,0.002044,0.008948,0.032027,3.951124e-06,1.000000e+00,0.008948,0.032027
2,0.776556,0.641285,0.806807,0.696077,0.000866,0.001474,2.417342e-06,2.792382e-05,0.000866,0.001474
3,0.185851,0.206495,0.291356,0.143201,0.017509,0.021748,6.443498e-06,2.109630e-06,0.017509,0.021748
4,0.131328,0.074609,0.165874,0.094106,0.003829,0.008871,1.211402e-05,8.881784e-16,0.003829,0.008871
5,0.716900,0.792928,0.555237,0.768846,0.000434,0.006954,2.381281e-05,1.000000e+00,0.000434,0.006954
6,1.027971,1.078430,1.000000,0.993269,0.030444,0.056772,2.456650e-05,1.092156e-04,0.030444,0.056772
7,0.001326,0.002699,0.000130,0.000838,0.000166,0.000164,4.686556e-06,0.000000e+00,0.000166,0.000164
8,1.048685,0.970317,0.941704,0.929672,0.043607,0.062707,2.061623e-05,2.081627e-04,0.043607,0.062707
9,0.960255,0.954974,0.975334,0.976934,0.034129,0.042986,2.953272e-05,2.822587e-03,0.034129,0.042986


In [91]:
X_2

,text_svd_cv,text_svd_tv,text_nmf_cv,text_nmf_tv,text_w2v,text_w2v_idf,text_ft,text_ft_idf,text_ft_norm,text_ft_norm_idf
0,0.806712,0.500128,0.838428,0.355612,0.000364,0.000508,1.630218e-06,1.403566e-05,1.0,0.000508
1,0.024767,0.124976,0.010350,0.058889,0.004470,0.015986,1.833234e-06,1.000000e+00,1.0,0.015986
2,0.439833,0.281440,0.345331,0.117332,0.000467,0.000607,1.706636e-06,1.523906e-05,1.0,0.000607
3,0.225016,0.242866,0.251424,0.332303,0.009360,0.008819,3.559627e-06,8.253448e-07,1.0,0.008819
4,0.115496,0.086889,0.206564,0.260241,0.002415,0.003125,5.168849e-06,4.440892e-16,1.0,0.003125
5,0.917953,0.929209,0.929328,0.916049,0.000326,0.002258,1.119201e-05,1.000000e+00,1.0,0.002258
6,1.028340,1.046117,0.998194,0.997277,0.022001,0.041478,1.156533e-05,3.980545e-05,1.0,0.041478
7,0.005548,0.015271,0.002373,0.001421,0.000085,0.000083,1.988674e-06,0.000000e+00,1.0,0.000083
8,1.026243,0.994580,0.985837,0.987525,0.026531,0.039072,9.790409e-06,4.219688e-05,1.0,0.039072
9,0.985374,0.890654,0.999902,0.980618,0.026225,0.034284,1.285395e-05,1.223512e-03,1.0,0.034284


## Классификация

In [47]:
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [41]:
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=42)

In [42]:
rfc = RandomForestClassifier(random_state=42)

In [43]:
grid = {'n_estimators': [5, 20, 70, 150],
       'max_depth': [1, 5, 10, None],
       'min_samples_leaf': [1, 10, 70, 150, 400],
       'criterion': ['gini', 'entropy']}

In [44]:
model = GridSearchCV(rfc, param_grid=grid, cv=5, n_jobs=-1, scoring='f1_micro', verbose=5)

In [45]:
model.fit(train_X, train_y)

Fitting 5 folds for each of 160 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   14.4s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   28.9s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:   56.8s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:  3.4min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [5, 20, 70, 150], 'max_depth': [1, 5, 10, None], 'min_samples_leaf': [1, 10, 70, 150, 400], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=5)

In [49]:
print('train', metrics.f1_score(train_y, model.predict(train_X), average='micro'))
print('test', metrics.f1_score(test_y, model.predict(test_X), average='micro'))

train 0.7472324723247232
test 0.5296070835639181


In [51]:
model.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'min_samples_leaf': 1,
 'n_estimators': 150}

### Другие параметры

Пересчитала все данные, увеличив длину векторов и количество признаков у моделей, чтобы не копировать весь код еще раз. Параметры для первой модели написаны в комментариях в соответствующих ячецках.

In [92]:
train_X_2, test_X_2, train_y_2, test_y_2 = train_test_split(X_2, y, random_state=42)

In [93]:
rfc_2 = RandomForestClassifier(random_state=42)

In [95]:
model_2 = GridSearchCV(rfc_2, param_grid=grid, cv=5, n_jobs=-1, scoring='f1_micro', verbose=5)

In [97]:
model_2.fit(train_X_2, train_y_2)

Fitting 5 folds for each of 160 candidates, totalling 800 fits


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   13.6s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   32.8s
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 800 out of 800 | elapsed:  4.6min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [5, 20, 70, 150], 'max_depth': [1, 5, 10, None], 'min_samples_leaf': [1, 10, 70, 150, 400], 'criterion': ['gini', 'entropy']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_micro', verbose=5)

In [98]:
print('train', metrics.f1_score(train_y_2, model.predict(train_X_2), average='micro'))
print('test', metrics.f1_score(test_y_2, model.predict(test_X_2), average='micro'))

train 0.7083025830258303
test 0.539568345323741


Стало чуть лучше

In [99]:
model_2.best_params_

{'criterion': 'gini',
 'max_depth': 10,
 'min_samples_leaf': 1,
 'n_estimators': 150}